# ReadMe

*train_s* - source train set (depending on domain, 57 or 58 scans)

*train_t* - target train set (depending on domain, 39 or 40 scans)

*val_s* - souce val set (2 scans)

*val_t* - target val set (10 scans)

*test_t* - target test set (10 scans)

Note that we do not use *train_t* set in this work, as we perform test-time domain adaptation 

# Generating the split

In [11]:
import numpy as np

from kswap.dataset.cc359 import CC359, scale_mri
from dpipe.dataset.wrappers import apply, cache_methods
from kswap.paths import DATA_PATH

from kswap.split import one2all_kspace_new

from pathlib import Path
from dpipe.io import save, PathLike, load

In [12]:
def build_split(folder, split, prefixes):
    folder = Path(folder)
    for i, ids in enumerate(split):
        local = folder / f'experiment_{i}'
        local.mkdir(parents=True)

        for val, prefix in zip(ids, prefixes):
            save(val, local / f'{prefix}_ids.json', indent=0)

In [13]:
# define the dataset
data_path = DATA_PATH
low = 0
high = 172

preprocessed_dataset = apply(CC359(data_path, low, high), load_image=scale_mri)
dataset = apply(cache_methods(apply(preprocessed_dataset, load_image=np.float16)), load_image=np.float32)

In [14]:
# define split
val_size_s = 2
val_size_t = 10
test_size = 10
domains = [0, 1, 2, 3, 4, 5]
n_folds = len(domains)

split = one2all_kspace_new(
    df=dataset.df,
    val_size_t=val_size_t,
    val_size_s=val_size_s,
    test_size=test_size,
    folds=domains,
    seed=42,
    mode='test_time'
)

prefixes=['train_s', 'train_t', 'val_s', 'val_t', 'test_t']

In [15]:
# build spllit
path = 'split'
build_split(path, split, prefixes)